## Collatz components notebook

In [9]:
"""
This notebook analyses core components of collatz 
sequences and their relationship: 
a.) k**i 
b.) beta = 1 + 1/(k*xi)
c.) alpha

The alphas of the sequence are compared with a predicted alphas values.
"""

# Fix possible import problems
import sys
sys.path.append("..")

# Imports
import random as rnd
from math import log2
import pandas as pd
from collatz import generator as gen
from collatz import commons as com

# Configuration
MAX_VALUE = 101
K_FACTOR = 3
MAX_ITERATIONS = 100
LOG_MODE = None
PRINT_TABLE = True

START_VALUE = rnd.randint(1, MAX_VALUE)

if START_VALUE % 2 == 0:
    START_VALUE = START_VALUE + 1

# START_VALUE = 13

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.expand_frame_repr', False)

# Create a collatz sequence and analyse it
analysis_frame = gen.generate_odd_collatz_sequence(
    start_value=START_VALUE, k=K_FACTOR, max_iterations=MAX_ITERATIONS)

# Drop last row if sequences has ended with 1
if len(analysis_frame) > 1 & int(analysis_frame.tail(1)["collatz"]) == 1:
    analysis_frame = analysis_frame[:-1]

# Derive new fields
analysis_frame["v1"] = START_VALUE
analysis_frame["n"] = analysis_frame.index + 1
analysis_frame["kn_log"] = log2(K_FACTOR) * analysis_frame["n"]
analysis_frame["beta"] = 1 + 1 / (K_FACTOR * analysis_frame["collatz"]) 
analysis_frame["beta_prod"] = analysis_frame["beta"].cumprod()

analysis_frame["alpha"] = analysis_frame["next_collatz"].apply(com.trailing_zeros)
analysis_frame["alpha"] = analysis_frame["alpha"].astype("int64")
analysis_frame["alpha_sum"] = analysis_frame["alpha"].cumsum()
analysis_frame["alpha_pred"] = (log2(K_FACTOR) * analysis_frame["n"]).astype('int64') + 1
analysis_frame["alpha_max"] = \
    log2(START_VALUE) + (analysis_frame["n"] * log2(K_FACTOR))
analysis_frame["alpha_max"] = analysis_frame["alpha_max"].astype('int64') + 1

analysis_frame["vi_hyp"] = (analysis_frame["v1"] * K_FACTOR**analysis_frame["n"])
analysis_frame["vi_hyp"] = analysis_frame["vi_hyp"] * analysis_frame["beta_prod"]
analysis_frame["vi_hyp"] = analysis_frame["vi_hyp"] / 2**analysis_frame["alpha_max"]

analysis_frame["beta_hyp"] = analysis_frame["alpha_pred"] - analysis_frame["kn_log"]

analysis_frame["vi_bin"] = analysis_frame["collatz"].apply(com.to_binary)
analysis_frame["vi+1_bin"] = analysis_frame["next_odd"].apply(com.to_binary)

analysis_frame["sigma"] = 1 / (K_FACTOR * analysis_frame["collatz"])
analysis_frame["sigma_sum"] = analysis_frame["sigma"].cumsum()

# Possible set log mode
if LOG_MODE:
    analysis_frame["v1"] = analysis_frame["v1"].apply(LOG_MODE)
    analysis_frame["collatz"] = analysis_frame["collatz"].apply(LOG_MODE)
    analysis_frame["next_odd"] = analysis_frame["next_odd"].apply(LOG_MODE)
    analysis_frame["beta"]= analysis_frame["beta"].apply(LOG_MODE)
    analysis_frame["beta_prod"]= analysis_frame["beta_prod"].apply(LOG_MODE)
    analysis_frame["beta_prod"]= analysis_frame["sigma"].apply(LOG_MODE)
    analysis_frame["beta_prod"]= analysis_frame["sigma_sum"].apply(LOG_MODE)
else:
    analysis_frame["beta_hyp"] = 2**analysis_frame["beta_hyp"]

# Validate alpha max & alpha pred
final_alpha = analysis_frame["alpha_sum"].max()
final_alpha_max = analysis_frame["alpha_max"].max()

alpha_max_valid = final_alpha == final_alpha_max

# Get max beta
beta_max = analysis_frame["beta_prod"].max()

# Print results
print_frame = analysis_frame[[
    "n", "v1", "collatz", "next_odd",
    "alpha", "alpha_sum", "alpha_pred", "alpha_max", 
    "beta", "beta_prod", "sigma", "sigma_sum"]]

print_frame.columns = [
    "n","v1", "vi", "vi+1",
    "a", "a_sum", "a_pred", "a_max", 
    "b", "b_prod", "s", "s_sum"]

print("Start value:", START_VALUE, 
      " K:", K_FACTOR, 
      "Beta max: ", round(beta_max, 4), " ",
       "Alphas valid:", alpha_max_valid, "\n")

if PRINT_TABLE:
    print(print_frame.to_string(index=False), "\n")


Start value: 65  K: 3 Beta max:  1.2294   Alphas valid: True 

 n  v1  vi vi+1  a  a_sum  a_pred  a_max         b    b_prod         s     s_sum
 1  65  65   49  2      2       2      8  1.005128  1.005128  0.005128  0.005128
 2  65  49   37  2      4       4     10  1.006803  1.011966  0.006803  0.011931
 3  65  37    7  4      8       5     11  1.009009  1.021083  0.009009  0.020940
 4  65   7   11  1      9       7     13  1.047619  1.069706  0.047619  0.068559
 5  65  11   17  1     10       8     14  1.030303  1.102121  0.030303  0.098862
 6  65  17   13  2     12      10     16  1.019608  1.123731  0.019608  0.118470
 7  65  13    5  3     15      12     18  1.025641  1.152545  0.025641  0.144111
 8  65   5    1  4     19      13     19  1.066667  1.229381  0.066667  0.210778 

